In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import faiss
import numpy as np

# Przykładowe dane
documents = [
    "Aby poprawnie wykonać przysiad, upewnij się, że twoje stopy są rozstawione na szerokość barków.",
    "Podczas wyciskania sztangi na ławce poziomej, upewnij się, że twoje łopatki są ściągnięte.",
    # Dodaj więcej dokumentów dotyczących ćwiczeń...
]

# Tokenizacja i wektoryzacja
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embeddings = []
for doc in documents:
    inputs = tokenizer(doc, return_tensors="pt", truncation=True, padding=True)
    embeddings.append(inputs['input_ids'].mean(dim=1).detach().numpy())

# Utworzenie indexu FAISS
index = faiss.IndexFlatL2(embeddings[0].shape[1])
index.add(np.vstack(embeddings))

# Model generacyjny
generator = pipeline("text2text-generation", model="facebook/bart-large-cnn")

def retrieve(query):
    query_embedding = tokenizer(query, return_tensors="pt", truncation=True, padding=True)['input_ids'].mean(dim=1).detach().numpy()
    D, I = index.search(query_embedding, k=1)
    return documents[I[0][0]]

def generate_answer(query):
    retrieved_doc = retrieve(query)
    input_text = f"Query: {query}\nDocument: {retrieved_doc}\nAnswer:"
    generated = generator(input_text, max_length=50, clean_up_tokenization_spaces=True)
    return generated[0]['generated_text']

# Przykładowe pytanie
question = "Jak poprawnie wykonywać przysiady?"
answer = generate_answer(question)
print(answer)


C:\Users\Damian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'faiss'

# load book from pdf file

In [24]:
import re

import PyPDF2

mypdf = open(".\\books\StartingStrength.pdf", mode='rb')
pdf_document = PyPDF2.PdfReader(mypdf)
len(pdf_document.pages)
first_page = pdf_document.pages[14]
print(first_page.extract_text())


# def fix_broken_words(text):
#     # Reguła do usunięcia niepotrzebnych spacji między literami na końcach linii
#     fixed_text = re.sub(r'(\w+)\s+\n\s*(\w+)', r'\1\2\n', text)
    
#     # Reguła do naprawy słów rozdzielonych spacją wewnątrz linii
#     fixed_text = re.sub(r'(\w+)\s+(\w+)', r'\1\2', fixed_text)

#     return fixed_text
    
# fixed_text = fix_broken_words(first_page.extract_text())

# print(fixed_text)

Basic  Barbell  Training 
The squa t has been the mos t importan t yet mos t poorl y understoo d exercis e in the trainin g 
arsena l for a very long time . Th e full rang e of motio n exercis e know n as the squa t is the singl e 
most usefu l exercis e in the weigh t room , and our mos t valuabl e tool for buildin g strength , power , 
and size (fig. 2-1) . 
The squa t is so effectiv e an exercis e becaus e of the way it uses the muscle s aroun d the core 
of the body . Muc h is mad e of core strength , and fortune s hav e been mad e sellin g new way s to 
train the core muscles . A correc t squa t perfectl y balance s all the force s aroun d the knee s and the 
hips, usin g thes e muscle s in exactl y the way the skeleta l biomechanic s are designe d for them to be 
used, ove r thei r anatomicall y full rang e of motion . Th e postura l muscle s of the lowe r back , the 
uppe r back , the abdominal s and latera l trun k muscles , the costa l (ribcage ) muscles , and eve n the 
shoul

In [41]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

# #converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir
# def convertMultiple(pdfDir, txtDir):
#     if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
#     for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
#         fileExtension = pdf.split(".")[-1]
#         if fileExtension == "pdf":
#             pdfFilename = pdfDir + pdf 
#             text = convert(pdfFilename) #get string of text content of pdf
#             textFilename = txtDir + pdf + ".txt"
#             textFile = open(textFilename, "w") #make text file
#             textFile.write(text) #write text to text file

# # set paths accordingly:
# pdfDir = "./books/"
# txtDir = "./books/"
# convertMultiple(pdfDir, txtDir)
pdf_file = ".\\books\StartingStrength.pdf"
text = convert(pdf_file)

In [42]:
print(text)

2nd Edition 
STARTING  STRENGTH 

BASIC  BARBELL  TRAINING 

Mark  Rippetoe 
&  Lon  Kilgore 

2nd Edition (revised) 

Copyright © 2007 by Mark Rippetoe and Lon Kilgore 

Editor - Stef Bradford, Ph.D. 

All rights reserved No pan of this publication may be reproduced, stored in a retrieval system or 
transmitted in a form by means, electronic, mechanical, photocopied, recorded, or otherwise without the 
poor written  consent of the publisher. The  authors  and publisher disclaim  any responsibility  for  any 
adverse effects or consequences  from  the misapplication  or injudicious use of the information presented 
in this text. 

ISBN  0-9768054-2-1 

Printed in  the United States of America  10  9  8  7 

The Aasgaard  Company 

3118 Buchanan, Wichita Falls, TX 76308, USA 

ii 

Contents 

Strength: Why and How .............................................................................................................. 2 

The Squat ...............................................

In [46]:
n = len(text)
print(n)

chunk_of_text = text[1000:10000]
print(chunk_of_text)

740650
................................................................................. 8 

The Bench Press ......................................................................................................................... 66 

The Deadlift ............................................................................................................................. 104 

The Press .................................................................................................................................. 148 

The Power Clean ......................................................................................................... 168 

Useful Assistance Exercises ....................................................................................................... 208 

Programming ......................................................................................................................................... 284 

Credits ..........................................

In [ ]:
import sparknlp

# Start Spark Session
spark = sparknlp.start()



In [47]:
from transformers import pipeline

# download model
qa_model = pipeline("question-answering")

# test model
question = "How old is Training for strength?"
context = chunk_of_text
qa_response = qa_model(question = question, context = context)
print(qa_response)

C:\Users\Damian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.



{'score': 0.8965405225753784, 'start': 1513, 'end': 1520, 'answer': 'insects'}
